In [2]:
# Imports
import zipfile
import os
import numpy as np
import csv
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Definindo o caminho da pasta de origem onde estão os arquivos a serem processados
pasta_origem = r'C:\Users\Pedro Carvalho\Desktop\Quanti\Arquivos_Diarios'  # Colocar a pasta onde seus dados estão

# Definindo o caminho da pasta de destino para onde os arquivos extraídos serão salvos
pasta_destino = r'C:\Users\Pedro Carvalho\Desktop\Arquivos_Diarios_Novo'  # Colocar a pasta para onde você quer que seus dados vão

# Verifica se a pasta de destino não existe. Caso ela não exista, o código cria essa pasta.
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

# Itera sobre todos os arquivos na pasta de origem
for arquivo in os.listdir(pasta_origem):
    # Verifica se o arquivo termina com a extensão .zip (ou seja, se é um arquivo ZIP)
    if arquivo.endswith('.zip'):
        # Cria o caminho completo para o arquivo ZIP na pasta de origem
        caminho_zip = os.path.join(pasta_origem, arquivo)
        
        # Abre o arquivo ZIP para leitura e o extrai para a pasta de destino
        with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
            zip_ref.extractall(pasta_destino)  # Extrai todo o conteúdo do ZIP para a pasta de destino

In [ ]:
# Lista todos os arquivos presentes na pasta de destino, após a extração
arquivos_extraidos = os.listdir(pasta_destino)

# Filtra a lista de arquivos extraídos, mantendo apenas os que têm a extensão '.csv'
arquivos_csv = [arquivo for arquivo in arquivos_extraidos if arquivo.endswith('.csv')]

# Itera sobre todos os arquivos CSV filtrados
for arquivo_csv in arquivos_csv:
    # Constrói o caminho completo para o arquivo CSV na pasta de destino
    caminho_csv = os.path.join(pasta_destino, arquivo_csv)
    
    # Lê o arquivo CSV em um DataFrame do pandas
    df = pd.read_csv(caminho_csv)
    
    # Exibe uma mensagem informando qual arquivo está sendo lido
    print(f"Lendo arquivo: {arquivo_csv}")
    
    # Exibe as primeiras 5 linhas do DataFrame lido (útil para verificar rapidamente o conteúdo)
    print(df.head())


In [5]:
# Inicializa uma lista vazia que vai armazenar os DataFrames
lista_dfs = []

# Itera sobre todos os arquivos CSV que foram filtrados anteriormente
for arquivo_csv in arquivos_csv:
    # Constrói o caminho completo para o arquivo CSV na pasta de destino
    caminho_csv = os.path.join(pasta_destino, arquivo_csv)
    
    # Lê o arquivo CSV em um DataFrame do pandas
    df = pd.read_csv(caminho_csv)
    
    # Adiciona o DataFrame lido à lista de DataFrames
    lista_dfs.append(df)

# Concatena todos os DataFrames da lista em um único DataFrame final
df_diarios_final = pd.concat(lista_dfs, ignore_index=True)


In [ ]:
# Definição da função que extrai a data e o valor de interesse em cada linha do DataFrame
def get_data_e_valor(row):
    # Verifica se a coluna 'FE Bid' não é nula. Se for válida, retorna a data e o valor de 'FE Bid'.
    if pd.notna(row['FE Bid']):
        return pd.Series([row['FE Bid Date'], row['FE Bid']])
    
    # Se 'FE Bid' não for válido, verifica 'Valuation NAV' e retorna seus dados se válido.
    elif pd.notna(row['Valuation NAV']): 
        return pd.Series([row['Valuation NAV Date'], row['Valuation NAV']])
    
    # Se 'Valuation NAV' também não for válido, tenta 'FE Generic NAV' e retorna seus dados se válido.
    elif pd.notna(row['FE Generic NAV']):
        return pd.Series([row['FE Generic NAV Date'], row['FE Generic NAV']])
    
    # Se nenhuma das anteriores for válida, tenta 'FE NAV' e retorna seus dados se disponível.
    elif pd.notna(row['FE NAV']):
        return pd.Series([row['FE NAV Date'], row['FE NAV']])
    
    # Caso contrário, tenta 'Transaction NAV' e retorna os dados correspondentes se for válido.
    elif pd.notna(row['Transaction NAV']):
        return pd.Series([row['Transaction NAV Date'], row['Transaction NAV']])
    
    # Se nenhuma das colunas anteriores tiver dados válidos, retorna NaN (vazio).
    else:
        return pd.Series([np.nan, np.nan])

# Aplica a função 'get_data_e_valor' para cada linha do DataFrame 'df_diarios_final'
# O resultado vai criar duas novas colunas: 'data' e 'valor', com base na função
df_diarios_final[['data', 'valor']] = df_diarios_final.apply(get_data_e_valor, axis=1)

# Filtra o DataFrame para selecionar colunas de interesse: 'ISIN', 'data' e 'valor'
# Remove as linhas com valores nulos e organiza o DataFrame por 'ISIN' e 'data' em ordem crescente
df_diarios_final_amostragem = df_diarios_final[['ISIN', 'data', 'valor']].dropna().sort_values(by=['ISIN', 'data'], ascending=[True, True]).head(250)

# Exibe as primeiras 250 linhas da amostra para verificação
print(df_diarios_final_amostragem)


In [11]:
df = pd.read_csv(r'C:\Users\Pedro Carvalho\Desktop\Quanti\Lote_Inicial\LoteInicial\LoteInicial.csv') # Colocar a pasta onde seus dados estão

df_filtered = df[df['Dynamic Data Type'] == 'FE Bid']

df_filtered = df_filtered.dropna(subset=['ISIN', 'General Reference Date', 'Dynamic Value'])

df_selected = df_filtered[['ISIN', 'General Reference Date', 'Dynamic Value']]

df_unique = df_selected.drop_duplicates(subset=['ISIN', 'General Reference Date'])

df_sorted = df_unique.sort_values(by=['General Reference Date', 'ISIN'], ascending=[True, True])

df_final = df_sorted.tail(250)

df_final.columns = ['ISIN', 'data', 'valor']

In [8]:
# Concatena dois DataFrames: 'df_diarios_final_amostragem' e 'df_final'
df_quanti_historico = pd.concat([df_diarios_final_amostragem, df_final], ignore_index=True).sort_values(by=['data'])


In [ ]:
print(df_quanti_historico)

In [14]:
# Salva o DataFrame 'df_quanti_historico' em um arquivo CSV chamado 'Historico_Quanti.csv'
df_quanti_historico.to_csv('Historico_Quanti12.csv', index=False)